In [1]:
# Insert the source code of the previous exercise into a loop that performs the comparison as the number of events considered for 
# the fit varies, from 20 to 10000, with a regular log-scale increment.
# 1) Use different plots to show the behavior of the parameters and their uncertainties as the number of events changes, 
#    for both types of estimators.
# 2) Add to the comparison the fit performed with the least squares method.
# 3) Which estimator is less biased at low statistics?

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from iminuit.cost import ExtendedBinnedNLL, UnbinnedNLL
from iminuit import Minuit
from IPython.display import display
import scipy.stats as sci

In [3]:
def model_function(x, mu, sigma):
    return sci.norm.pdf(x,mu,sigma)

def cdf_model_function(bin_edges, mu, sigma):
    return sci.norm.cdf(bin_edges,mu,sigma)

In [ ]:
true_mu = 0
true_sigma = 1
N = 100000
data = np.random.normal(true_mu,true_sigma,N)

In [ ]:
# BINNED FIT
bin_content, bin_edges = np.histogram(data, int(np.ceil(1+3.322*np.log(N)))) # number of event per bin
my_cost_binned_func = ExtendedBinnedNLL(bin_content, bin_edges, cdf_model_function)
my_minuit_binned = Minuit(my_cost_binned_func, mu = np.mean(data), sigma = np.std(data))
my_minuit_binned.limits["mu","sigma"]=(0,None)
my_minuit_binned.migrad()
print (f'Convergenza del fit: {my_minuit_binned.valid}')
display(my_minuit_binned)

print ('Associated binned p-value: ', 1. - sci.chi2.cdf (my_minuit_binned.fval, df = my_minuit_binned.ndof))

# UNBINNED FIT
my_cost_unbinned_func = UnbinnedNLL(data, model_function)
my_minuit_unbinned = Minuit(my_cost_unbinned_func, mu = np.mean(data), sigma = np.std(data))
my_minuit_unbinned.limits["mu","sigma"]=(0,None)
my_minuit_unbinned.migrad()
print (f'Convergenza del fit: {my_minuit_unbinned.valid}')
display(my_minuit_unbinned)

print ('Associated unbinned p-value: ', 1. - sci.chi2.cdf (my_minuit_unbinned.fval, df = my_minuit_unbinned.ndof))

# ------------------------------------------------------------------------------------------------- #

# PLOT
x_coord = np.linspace(np.min(data), np.max(data), 10000)
y_f_unb = []
y_f = []
for i in range(len(x_coord)):
    y_f_unb.append(model_function(x_coord[i], my_minuit_unbinned.values[0], my_minuit_unbinned.values[1]))
    y_f.append(model_function(x_coord[i], my_minuit_binned.values[0], my_minuit_binned.values[1]))

fig, ax = plt.subplots(1,1)
ax.hist(data, bins=bin_edges, color='lightblue', label='Data', edgecolor='blue', alpha=0.6, density=True)
ax.plot(x_coord, y_f, color='red', label='Fit')
ax.plot(x_coord, y_f_unb, color='red', label='Fit')
ax.set_xlabel('Measures')
ax.set_ylabel('N')
ax.set_title('Fit')
ax.legend()
ax.grid(color='black', linestyle='--', linewidth=0.5, alpha=0.5)
    
plt.show()

In [ ]:
# UNBINNED FIT
my_cost_unbinned_func = UnbinnedNLL(data, model_function)
my_minuit_unbinned = Minuit(my_cost_unbinned_func, mu = np.mean(data), sigma = np.std(data))
my_minuit_unbinned.limits["mu","sigma"]=(0,None)
my_minuit_unbinned.migrad()
print (f'Convergenza del fit: {my_minuit_unbinned.valid}')
display(my_minuit_unbinned)

print ('Associated p-value: ', 1. - sci.chi2.cdf (my_minuit_unbinned.fval, df = my_minuit_unbinned.ndof))

# PLOT
x_coord = np.linspace(np.min(data), np.max(data), 10000)
y_f_unb = []
for i in range(len(x_coord)):
    y_f_unb.append(model_function(x_coord[i], my_minuit_unbinned.values[0], my_minuit_unbinned.values[1]))

fig, ax = plt.subplots(1,1)
ax.hist(data, bins=bin_edges, color='lightblue', label='Data', edgecolor='blue', alpha=0.6, density=True)
ax.plot(x_coord, y_f_unb, color='red', label='Fit')
ax.set_xlabel('Measures')
ax.set_ylabel('N')
ax.set_title('Fit')
ax.legend()
ax.grid(color='black', linestyle='--', linewidth=0.5, alpha=0.5)
    
plt.show()